# Lab 07: Generative Adversarial Networks (GAN)
Trong bài thực hành này:
- Xậy dựng và huấn luyện một hệ GAN trên tập data MNIST

Reference:
- Generative Adversarial Networks (GAN), https://arxiv.org/abs/1406.2661
- Deep Convolutional GAN (DCGAN), https://arxiv.org/pdf/1511.06434

## Xây dựng cấu trúc discriminator và generator

In [ ]:
## import các module cần thiết
import tensorflow as tf
from tensorflow.keras.layers import Convolution2D, Dense, Flatten, Input, Reshape, Dropout, ReLU, Conv2DTranspose, BatchNormalization, LeakyReLU
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
from tensorflow.keras import layers


## Xây dựng cấu trúc cho discriminator
def get_discriminator():
    """
    Input: ảnh (28, 28), các phần tử trong ảnh thuộc  [-1,1]
    Output: số thực trong đoạn [0,1], càng gần 1 thì ảnh input càng là ảnh thật không phải do generator tạo ra
    """
    inputs = Input(shape=(28,28))
    xx = Reshape((28,28,1))(inputs)
    xx = Convolution2D(filters=64,
                       kernel_size=[5,5],
                       strides=[2,2],
                       padding='same')(xx)
    xx = LeakyReLU(0.2)(xx)

    xx = Convolution2D(filters=128,
                       kernel_size=[5,5],
                       strides=[2,2],
                       padding='same')(xx)
    xx = LeakyReLU(0.2)(xx)

    xx = Flatten()(xx)
    outputs = Dense(units=1, activation='sigmoid')(xx)

    model = Model(inputs=inputs, outputs=outputs)

    return model

## Xây dựng cấu trúc cho generator
def get_generator():
    """
    Input: vector (100,) 
    Output: ảnh (28,28) được tạo từ vector input
    """
    inputs = Input(shape=(100,))
    xx = Dense(units=7*7*256,
               use_bias=False)(inputs)
    xx = Reshape((7,7,256))(xx)
    xx = BatchNormalization()(xx)
    xx = LeakyReLU(0.2)(xx)
    
    xx = Conv2DTranspose(filters=128,
                         kernel_size=[5,5],
                         strides=[1,1],
                         padding='same',
                         use_bias=False)(xx)
    xx = BatchNormalization()(xx)
    xx = LeakyReLU(0.2)(xx)

    xx = Conv2DTranspose(filters=64,
                         kernel_size=[5,5],
                         strides=[2,2],
                         padding='same',
                         use_bias=False)(xx)
    xx = BatchNormalization()(xx)
    xx = LeakyReLU(0.2)(xx)

    xx = Conv2DTranspose(filters=1,
                         kernel_size=[5,5],
                         strides=[2,2],
                         padding='same',
                         use_bias=False,
                         activation='tanh')(xx)   ## hàm kích hoạt lớp cuối cùng là tanh, giá trị điểm ảnh thuộc [-1, 1]

    outputs=Reshape((28,28))(xx)

    model = Model(inputs=inputs, outputs=outputs)

    return model


## Thiết lập quá trình huấn luyện GAN

In [ ]:
## Thiết lập quá trình training cho generator
def get_generator_training(discriminator, generator):
    """
    Input: discriminator, generator
    Output: một model, thiết lập sẵn quá trình huấn luyện cho generator
    """

    # khi huấn luyện generator, vector được đưa vào generator tạo ra ảnh, ảnh đó sẽ tiếp tục được đưa vào discriminator
    inputs = generator.inputs                       # input của quá trình là input của generator
    outputs = discriminator(generator.outputs)      # output của generator được cho vô discriminator
    model = Model(inputs=inputs, outputs=outputs)   

    # discriminator được đóng băng trong quá trình huấn luyện generator
    generator.trainable = True
    discriminator.trainable = False

    # compile
    model.compile(optimizer=tf.keras.optimizers.Adam(2e-4, beta_1=0.5),
                  loss=tf.keras.losses.binary_crossentropy,
                  metrics=['accuracy'])
    
    return model

## Thiết lập quá trình train cho discriminator
def get_discriminator_training(discriminator):
    """
    Input: discriminator
    Output: một model, thiết lập sẵn quá trình huấn luyện cho discriminator
    """

    ## model này có input y chang discriminator
    inputs = discriminator.inputs
    outputs = discriminator.outputs
    model = Model(inputs=inputs, outputs=outputs)  

    ## discriminator được huấn luyện
    discriminator.trainable = True

    ## compile
    model.compile(optimizer=tf.keras.optimizers.Adam(2e-4, beta_1=0.5),
                  loss=tf.keras.losses.binary_crossentropy,
                  metrics=['accuracy'])
    return model


## Huấn luyện GAN

In [ ]:
## tải MNIST dataset từ keras
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
##resacle ảnh thành ảnh thực trong đoạn [0,1]
X_train, X_test = (X_train-127.5)/127.5, (X_test-127.5)/127.5

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

## một hàm tạo ảnh ngẫu nhiên rồi vẽ ra
def generate_samples(generator, seed=None):
    if seed is None:
        seed = np.random.normal(0.0, 1.0, size=[16,100])    ## tạo 16 vector ngẫu nhiên
    generated_images = generator.predict(seed)   ## tạo ảnh

    ## vẽ từng ảnh được tạo
    for i, image in enumerate(generated_images):
        plt.subplot(4,4,i+1)
        plt.imshow(image, cmap='gray', vmin=-1.0, vmax=1.0)
        plt.axis('off')
    plt.show()

In [ ]:
## tạo các object
generator = get_generator()
discriminator = get_discriminator()
generator_train = get_generator_training(discriminator, generator)
discriminator_train = get_discriminator_training(discriminator)

generator.summary()
discriminator.summary()


In [ ]:
seed = np.random.normal(0.0, 1.0, size=[16,100])     ## seed được dùng để tạo ảnh vẽ ra trong quá trình huấn luyện
generate_samples(generator, seed)                   ## thử tạo ảnh khi generator mới tạo

In [ ]:
### Huấn luyện
batch_size = 200

## thiết lập các labels sẵn
y_ones = np.ones((batch_size,))
y_zeros = np.zeros((batch_size,))
y_train_batch = np.concatenate([0.9*y_ones, y_zeros], axis=0)
y_eval_zeros = np.zeros((10000,))

for epoch in range(100):
    np.random.shuffle(X_train)    ## xáo vi vị trí các ảnh trong tập train

    for i_batch in range(0,60000,batch_size):
        ## huấn luyện discriminator

        X_real_batch = X_train[i_batch:i_batch+batch_size]  ## lấy một batch ảnh thật

        random_noise_batch = np.random.normal(0.0, 1.0, size=[batch_size,100])  ## tạo một batch vector ngẫu nhiên
        X_fake_batch = generator.predict(random_noise_batch)                    ## tạo một batch các ảnh giả từ generator

        X_train_batch = np.concatenate([X_real_batch, X_fake_batch], axis=0)    ## nối ảnh thật và ảnh giả
        
        discriminator_train.train_on_batch(X_train_batch, y_train_batch)        ## huấn luyện discriminator với label của ảnh thật là 1.0, label của ảnh giả là 0.0

        ## huấn luyện generator
        random_noise_batch = np.random.normal(0.0, 1.0, size=[batch_size,100])  ## tạo một batch vector ngẫu nhiên
        generator_train.train_on_batch([random_noise_batch], y_ones)            ## huấn luyện generator, với label toàn là 1.0 (để lừa discriminator rằng ảnh giả này là ảnh thật)
    
    ##xem accuracy của discriminator với ảnh giả là bao nhiêu
    random_noise = np.random.normal(0.0, 1.0, size=[10000,100])
    _, train_acc = generator_train.evaluate([random_noise], y_eval_zeros, verbose=0)

    print("Epoch {} - Discriminator accuracy on fake images {}".format(epoch+1, train_acc ))

    ## tạo thử ảnh từ seed xem nó có đẹp hơn không
    if epoch % 1 == 0:
        print("Generated images after epoch {}".format(epoch))
        generate_samples(generator, seed)


## Bài tập

- Đọc bài Conditional Generative Adversarial Nets, https://arxiv.org/abs/1411.1784
- Xây dựng và huấn luyện một hệ Conditional DCGAN
- Tạo 200 ảnh từ generator bằng code phía dưới

```python
seed = ##???
generated_images = generator.predict(seed)

plt.figure(figsize=(20,10))

for i, image in enumerate(generated_images):
    plt.subplot(10,20,i+1)
    plt.imshow(image, cmap='gray', vmin=-1.0, vmax=1.0)
    plt.axis('off')
plt.show()
```


<img src="generated_images_ConditionalDCGAN.png" width="60%" height="60%">

